# Computation of Granularity Adjustments of MDB Portfolios

In [320]:
import numpy as np
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt 
import pandas as pd
import copy
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%aimport IS_misc_functions
from IS_misc_functions import *
from misc_functions import *
from actuarial_training import *
from MtM_training import *
from scipy.optimize import minimize
import json
import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler,normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create Portfolios

Read in everything

In [321]:
by_country = pd.read_excel("MDB portfolios.xlsx", sheet_name='by country')
maturity = pd.read_excel("MDB portfolios.xlsx", sheet_name='maturity')
summary = pd.read_excel("MDB portfolios.xlsx", sheet_name='summary of current info')
comments = pd.read_excel("MDB portfolios.xlsx", sheet_name='comments')
rating = pd.read_excel("MDB portfolios.xlsx", sheet_name='rating')
transition_matrix_SP = pd.read_csv("transition_matrix_SP.csv",delimiter = ",",index_col = 0)
transition_matrix_RC = pd.read_csv("transition_matrix_RC.csv",delimiter = ";",index_col = 0)

Preprocessing the Data

In [322]:
SP_dict = trans_matrix_to_dict(transition_matrix_SP)
transition_matrix_SP

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,Cs,D
AAA,96.79,2.71,0.42,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00
AA+,6.45,85.16,6.61,1.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AA,0.00,6.22,85.17,6.74,0.52,0.42,0.10,0.52,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00
AA-,0.00,0.00,7.82,83.45,7.16,0.17,0.50,0.17,0.00,0.17,0.44,0.00,0.00,0.11,0.00,0.00,0.00,0.00
A+,0.00,0.00,0.07,13.35,73.28,9.30,2.03,1.12,0.14,0.63,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.01
A,0.00,0.00,0.00,1.15,12.33,77.29,5.71,1.68,0.77,0.96,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.01
A-,0.00,0.00,0.00,0.00,0.94,11.47,77.82,6.94,0.41,1.57,0.67,0.16,0.00,0.00,0.00,0.00,0.00,0.02
BBB+,0.00,0.00,0.00,0.00,0.00,2.16,12.39,70.86,11.24,2.41,0.60,0.24,0.06,0.00,0.00,0.00,0.00,0.04
BBB,0.00,0.00,0.00,0.00,0.00,0.00,1.87,16.60,68.05,11.16,0.99,0.11,0.00,0.50,0.22,0.11,0.33,0.06
BBB-,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.93,14.94,74.69,6.50,2.13,0.27,0.08,0.15,0.12,0.08,0.11


In [323]:
RC_dict = trans_matrix_to_dict(transition_matrix_RC)
transition_matrix_RC

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,Cs,D
AAA,96.79,2.71,0.42,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00
AA+,6.45,85.16,6.61,1.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AA,0.00,6.22,85.17,6.74,0.52,0.42,0.10,0.52,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00
AA-,0.00,0.00,7.82,83.45,7.16,0.17,0.50,0.17,0.00,0.17,0.44,0.00,0.00,0.11,0.00,0.00,0.00,0.00
A+,0.00,0.00,0.07,13.35,73.28,9.30,2.03,1.12,0.14,0.63,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A,0.00,0.00,0.00,1.15,12.33,77.30,5.71,1.68,0.77,0.96,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A-,0.00,0.00,0.00,0.00,0.94,11.47,77.83,6.94,0.41,1.57,0.67,0.16,0.00,0.00,0.00,0.00,0.00,0.01
BBB+,0.00,0.00,0.00,0.00,0.00,2.16,12.40,70.87,11.25,2.41,0.60,0.24,0.06,0.00,0.00,0.00,0.00,0.01
BBB,0.00,0.00,0.00,0.00,0.00,0.00,1.87,16.61,68.08,11.17,0.99,0.11,0.00,0.50,0.22,0.11,0.33,0.02
BBB-,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.93,14.95,74.75,6.51,2.13,0.27,0.08,0.15,0.12,0.08,0.03


In [324]:
#Small Tolerance to avoid 0s
tolerance = 10e-10

# list of banks
bank_names = list(summary.iloc[:,0])

# Exclude EIB and NIB from being considered
bank_names = [b for b in bank_names if b != "EIB" and b!= "NIB"]


# SP Rating of Countries
country_sp = {rating["Country"][i].lower().replace(" ", ""): 
              str(rating["sp_cb"][i]).replace(" ", "")  for i in range(len(rating["sp_cb"]))}

def create_PD_dict(transition_matrix,unrated_b_minus = True):
    country_dict = country_sp #copy.copy(country_sp)

    # The PDs of Rating Classes
    Rating_PD = {transition_matrix.index[i]: np.maximum(transition_matrix["D"][i],tolerance)  for i in range(len(transition_matrix.index))}

    # Companies with D rating are not in default wrt other MDBs:
    Rating_PD["D"] = Rating_PD["Cs"]
    Rating_PD["SD"] = Rating_PD["Cs"]
    
    if unrated_b_minus:
        for country_rating in set(country_dict.values()):
            if country_rating in ["C","CC","CCC-","CCC","CCC+"]:
                Rating_PD[country_rating] = Rating_PD["Cs"]
            if country_rating in ['0',0]:
                Rating_PD[country_rating] = Rating_PD["B-"] 

#         for country_rating in set(country_dict.values()):
#             if country_rating not in Rating_PD.keys():
#                 Rating_PD[country_rating] = "nan"

        for country in country_dict:
            if country_dict[country] in ["C","CC","CCC-","CCC","CCC+"]:
                country_dict[country] = "Cs"
            if country_dict[country] in ['0',0]:
                country_dict[country] = "B-"
    
    elif unrated_b_minus == False:

        # Adjust the PDs further 
        for country_rating in set(country_dict.values()):
            if country_rating in ["C","CC",'0',0,"CCC-","CCC","CCC+"]:
                Rating_PD[country_rating] = Rating_PD["Cs"] 

        for country_rating in set(country_dict.values()):
            if country_rating not in Rating_PD.keys():
                Rating_PD[country_rating] = "nan"

        for country in country_dict:
            if country_dict[country] in ["C","CC",'0',0,"CCC-","CCC","CCC+"]:
                country_dict[country] = "Cs"

    # Assign the ratings to the countries
    countries_PD = {country.lower().replace(" ", ""): Rating_PD[country_dict[country]] for country in country_dict.keys()}
    return countries_PD

In [325]:
countries_PD_SP = create_PD_dict(transition_matrix_SP)
countries_PD_RC = create_PD_dict(transition_matrix_RC)

Diplay the used PDs

In [326]:
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.DataFrame([countries_PD_SP,countries_PD_RC],index =["S&P PDs","RC PDs"]).transpose())

,S&P PDs,RC PDs
afghanistan,51.47,14.70
albania,1.46,0.42
algeria,0.90,0.26
andorra,0.04,0.01
angola,7.59,2.17
anguilla,7.59,2.17
antiguaandbarbuda,51.47,14.70
argentina,51.47,14.70
armenia,0.90,0.26
aruba,0.06,0.02


Find the spaces between banks

In [327]:
spaces = np.where(pd.isnull(by_country.iloc[:,3]))[0]
spaces[0] = 1
spaces = np.append(spaces,len(by_country.iloc[:,3]))

### Create the portfolios

Define a function to create the portfolios

In [328]:
def create_portfolio(q = 0.99, ELGD_val = 0.45,
                     transition_matrix = transition_matrix_SP,
                     countries_PD = countries_PD_SP,
                    constant_rho = False,
                    rho = 0.35):
    portfolios = {}
    for i in range(len(bank_names)):
        considered_countries = np.array(by_country.iloc[:,1][(spaces[i]+1):(spaces[i+1]-1)])
        PD = []
        EAD = []
        g = []
        for j in range(len(considered_countries)):
            country = considered_countries[j].lower().replace(" ", "")
            if country in countries_PD.keys() and countries_PD[country] != 'nan': # nur anlegen wenn existent
                PD.append(countries_PD[country] /100)
                EAD.append(by_country.iloc[:,3][(spaces[i]+1)+j])
                g.append(S-list(transition_matrix.index).index(country_sp[country]))
        EAD = EAD/np.sum(np.array(EAD,dtype = np.float64))
        PD = np.array(PD)
        ELGD = np.array([ELGD_val]*len(EAD))
        if constant_rho:
            def to_solve(om):
                b = (0.11852-0.05478*np.log(PD))**2
                R_corr = rho
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                alpha_X =  gamma.ppf(q,a =0.25, scale = 1/0.25)
                return  (np.abs(ELGD*PD*om*(alpha_X-1)-K))
        else:
            def to_solve(om):
                b = (0.11852-0.05478*np.log(PD))**2
                R_corr = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50)))
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                alpha_X =  gamma.ppf(q,a =0.25, scale = 1/0.25)
                return  (np.abs(ELGD*PD*om*(alpha_X-1)-K))
        omega = fsolve(to_solve,[0.25]*len(EAD))
        omega = np.minimum(omega,1)
        #bnds = [(0,1) for i in range(len(EAD))]
        #omega = minimize(to_solve,[1]*len(EAD), bounds=bnds).x
        portfolios[bank_names[i]] = [EAD,ELGD,PD,omega,g]
    return portfolios

Create the portfolios

In [329]:
portfolios_0999 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP)

# Test everything

First load the models

In [330]:
# Actuarial 
model_actuarial_0999 = keras.models.load_model("model_actuarial_0999.h5")
model_actuarial_0999.compile()


#Mark to Market
model_mtm_0999 = keras.models.load_model("model_mtm_0999.h5")
model_mtm_0999.compile()




In [331]:
# # Read Data
def read_data(xpath,ypath):
    df_x = pd.read_csv(xpath, header=None)
    df_x = df_x.dropna()
    df_x = df_x.iloc[:,1:]

    df_y = pd.read_csv(ypath, header=None)
    df_y = df_y.dropna()
    df_y = df_y.iloc[:,1:]
    
    return df_x,df_y
x,y = read_data("csv/X_mtm_1.csv","csv/Y_mtm_1.csv")
for i in range(2,11):
    x_new,y_new = read_data("csv/X_mtm_"+str(i)+".csv","csv/Y_mtm_"+str(i)+".csv")
    x = pd.concat([x,x_new])
    y = pd.concat([y,y_new])
x_train = x.reset_index(drop = True)
y_train = y.reset_index(drop = True)

x_test, y_test = read_data("csv/X_mtm_test.csv","csv/Y_mtm_test.csv")

# Reduce Dimension of the data
def reduce_dim_mtm(x):
    ELGD_new = x.iloc[:,0]
    EAD_new = x.iloc[:,100:200]
    rho_new = x.iloc[:,200:300]
    c_new = x.iloc[:,300:400]
    g_new = x.iloc[:,400:500]
    D_new = x.iloc[:,500]
    return pd.concat([ELGD_new,EAD_new,rho_new,c_new,g_new,D_new],axis = 1)

x_train_small = reduce_dim_mtm(x_train)
x_test_small = reduce_dim_mtm(x_test)

Define a Function for the computations

In [332]:
transition_matrix = pd.read_csv("transition_matrix_SP.csv",delimiter = ",")
PDs = np.maximum(np.array(transition_matrix["D"])/100,1e-8)
SP_dict = trans_matrix_to_dict(transition_matrix)
with open("probs.txt") as tweetfile:
    probs = json.load(tweetfile)
probs = {float(k):float(v) for k,v in probs.items()}
PDs =  np.maximum(np.array([float(k) for k,v in probs.items()])/100,1e-10)
PD_distribution = np.array([float(v) for k,v in probs.items()])
S = 17
g_bounds = 2,S
g_vals = np.arange(g_bounds[0],g_bounds[1])
g_probs = np.flip(np.concatenate([[0.0001,0.0001],PD_distribution]))
g_probs = g_probs/np.sum(g_probs)


In [333]:
def compute_GA_MC(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               trans_dict = SP_dict,
               compute_mtm=True,
               constant_rho = False,
               constant_rho_val = 0.35,
               relative_to_UL = False,
              n_mc = 1000000, # For Actuarial Approach
              n_mc_mtm = 1000000,
              n_mc_mtm_n1 = 50,
              maturity = np.array([1]*len(portfolios_0999)),
              xi = 0.25,
              coupons = 0.01):
    GA_MtM = []
    GA_actuarial = []
    
    i = -1
    for portfolio in portfolios:
        i = i +1
        EAD,ELGD,PD,omega,g = portfolios[portfolio]
        ELGD = np.array([ELGD_const]*len(EAD))
        if len(EAD)>100: # Choose 100 indices
            indices = np.random.choice(range(len(EAD)-1), 100,replace = False)
            EAD = EAD[indices]
            PD = PD[indices]
            omega = omega[indices]
            ELGD = ELGD[indices]
            g = np.array(g)[indices]
        EAD = EAD/np.sum(EAD)
        # Adjust Input for NNs:
        rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50)))
        rho = np.reshape(np.append(rho,[0]*(100-len(EAD))),(1,100))
        c = np.array([coupons]*len(EAD))
        c = np.reshape(np.append(c,[0]*(100-len(EAD))),(1,100))
        g = np.array(g)
        g = np.reshape(np.append(g,[0]*(100-len(EAD))),(1,100))
        D= np.array([1]*len(EAD))
        D=  np.reshape(np.append(D,[0]*(100-len(EAD))),(1,100))
        EAD = np.reshape(np.append(EAD,[0]*(100-len(EAD))),(1,100))
        ELGD = np.reshape(np.append(ELGD,[0]*(100-len(ELGD))),(1,100))
        PD = np.reshape(np.append(PD,[0]*(100-len(PD))),(1,100))
        omega = np.reshape(np.append(omega,[0]*(100-len(omega))),(1,100))
        Input = np.concatenate([ELGD ,EAD ,PD ,omega ],axis =1)
            
        GA_MtM.append(MC_IS_MtM(ELGD[0,:],EAD[0,:],rho[0,:],c[0,:],g[0,:],
                               SP_dict,D[0,:],r_nelson,
                               q=0.999,n = n_mc))
        GA_actuarial.append(MC_IS_actuarial(np.reshape(ELGD[0,:],(1,100)),
                                                 np.reshape(EAD[0,:],(1,100)),
                                                np.reshape(PD[0,:],(1,100)),
                                                np.reshape(omega[0,:],(1,100)),
                                                 q=0.999,n = n_mc)[0])       
        
        
    df = pd.DataFrame([[(1/100)*len(portfolios[portfolio][0]) for portfolio in portfolios],
                          GA_MtM, GA_actuarial],
                         columns=bank_names, 
                         index = ["Borrowers",
                                  "GA MtM MC",
                                  "GA CR+ MC"])
    return df*100
                

In [334]:
MC_045 = compute_GA_MC(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
              n_mc = 10, # For Actuarial Approach
              n_mc_mtm = 10,
              n_mc_mtm_n1 = 50)
MC_045.iloc[1] = np.array([18.71,10.34,12.48,13.32,18.76,25.85,36.44,6.38,25.95,22.75,11.20])
MC_045.iloc[2] = np.array([12.46,7.07,8.00,9.24,11.89,20.38,27.13,4.35,15.23,15.78,8.76])
MC_045

,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.00,38.00,29.00,26.00,16.00,11.00,4.00,78.00,21.00,8.00,37.00
GA MtM MC,18.71,10.34,12.48,13.32,18.76,25.85,36.44,6.38,25.95,22.75,11.20
GA CR+ MC,12.46,7.07,8.00,9.24,11.89,20.38,27.13,4.35,15.23,15.78,8.76


In [335]:
MC_01 = compute_GA_MC(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
                          n_mc = 10, # For Actuarial Approach
              n_mc_mtm = 10,
              n_mc_mtm_n1 = 50)
MC_01.iloc[1] = np.array([11.58,7.78,8.09,10.19,11.72,18.88,22.12,3.74,15.98,13.39,6.84])
MC_01.iloc[2] = np.array([4.35,2.65,3.13,3.38,3.96,5.98,12.72,1.49,6.88,5.58,2.33])
MC_01

,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.00,38.00,29.00,26.00,16.00,11.00,4.00,78.00,21.00,8.00,37.00
GA MtM MC,11.58,7.78,8.09,10.19,11.72,18.88,22.12,3.74,15.98,13.39,6.84
GA CR+ MC,4.35,2.65,3.13,3.38,3.96,5.98,12.72,1.49,6.88,5.58,2.33


import scalers

In [336]:
scaler_actuarial = joblib.load('scaler_actuarial.gz')
scaler_mtm = joblib.load('scaler_mtm.gz')

In [337]:
def compute_GA(q=0.99,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999,
               trans_dict = SP_dict,
               compute_mtm=True,
               constant_rho = False,
               constant_rho_val = 0.35,
               relative_to_UL = False,
              n_mc = 100000, # For Actuarial Approach
              n_mc_mtm = 100000,
              n_mc_mtm_n1 = 50,
              maturity = np.array([1]*len(portfolios_0999)),
              xi = 0.25,
              coupons = 0.01):
    GA_NN_approx = []
    GA_approx_list = []
    GA_NN_approx_MtM = []
    GA_approx_list_MtM = []
    
    i = -1
    for portfolio in portfolios:
        i = i +1
        EAD,ELGD,PD,omega,g = portfolios[portfolio]
        ELGD = np.array([ELGD_const]*len(EAD))
        if len(EAD)>100: # Choose 100 indices
            indices = np.random.choice(range(len(EAD)-1), 100,replace = False)
            EAD = EAD[indices]
            PD = PD[indices]
            omega = omega[indices]
            ELGD = ELGD[indices]
            g = np.array(g)[indices]
        EAD = EAD/np.sum(EAD)
        # Adjust Input for NNs:
        rho = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50)))
        rho = np.reshape(np.append(rho,[0]*(100-len(EAD))),(1,100))
        c = np.array([coupons]*len(EAD))
        c = np.reshape(np.append(c,[0]*(100-len(EAD))),(1,100))
        g = np.array(g)
        g = np.reshape(np.append(g,[0]*(100-len(EAD))),(1,100))
        D= np.array([1]*len(EAD))
        D=  np.reshape(np.append(D,[0]*(100-len(EAD))),(1,100))
        EAD = np.reshape(np.append(EAD,[0]*(100-len(EAD))),(1,100))
        ELGD = np.reshape(np.append(ELGD,[0]*(100-len(ELGD))),(1,100))
        PD = np.reshape(np.append(PD,[0]*(100-len(PD))),(1,100))
        omega = np.reshape(np.append(omega,[0]*(100-len(omega))),(1,100))
        Input = np.concatenate([ELGD ,EAD ,PD ,omega ],axis =1)
            
        GA_approx = GA_GL(PD[0,:][EAD[0,:]>0],
                              ELGD[0,:][EAD[0,:]>0],
                              EAD[0,:][EAD[0,:]>0], 1, # Vector of Maturities
                                  q = 0.999, # quantile level
                                  xi = 0.25, # precision parameter,
                                  nu = 0.25,# recovery parameter
                                  rho = omega[0,:][EAD[0,:]>0],
                                  LGD_constant = False)

        
        Input_2 = np.concatenate([Input,np.reshape(GA_approx,(1,1))],axis = 1)
        GA_NN_approx.append(model_actuarial.predict(scaler_actuarial.transform(Input_2))[0][0])
        GA_approx_list.append(GA_approx)
        
        # MtM
        Input_MtM = np.concatenate([ELGD,EAD,rho,c,g,D],axis =1)
        GA_approx_mtm = GA_GM(PD = 0, ELGD =ELGD[0,:][EAD[0,:]>0],
                                   A = EAD[0,:][EAD[0,:]>0], M= D[0,:][EAD[0,:]>0],
                      q=0.999,        
                     r = r_nelson,
                     T=1,
                     g= np.array(g[0,:][EAD[0,:]>0]),
                     trans_prob=SP_dict["trans_prob"],#N times S matrix
                     psi = 0.4, # Market Sharpe Ratio,
                     rho = rho[0,:][EAD[0,:]>0],
                     c = c[EAD>0],
                    nu =0.25,               
                     S = S, # Number of States of Rating
                     default_only = False,
                      LGD_constant = False)

        
        Input_mtm_2 = np.concatenate([Input_MtM,np.reshape(GA_approx_mtm,(1,1))],axis = 1)
        GA_NN_approx_MtM.append(model_mtm.predict(scaler_mtm.transform(Input_mtm_2))[0][0])
        GA_approx_list_MtM.append(GA_approx_mtm)

        
        df = pd.DataFrame([[(1/100)*len(portfolios[portfolio][0]) for portfolio in portfolios],
                              GA_NN_approx, GA_approx_list,GA_NN_approx_MtM,GA_approx_list_MtM],
                             columns=bank_names, 
                             index = ["Borrowers",
                                      "GA CR+ NN",
                                     "GA CR+ approx",
                                     "GA MtM NN",
                                     "GA MtM approx"])
    return df*100
                

# GA Computations for S&P Transition Matrix

In [338]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 1.) q = 99.9%, ELGD = 0.45

In [339]:
df_999 = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999)
# Monte Carlo Results
df_2 = pd.DataFrame([MC_045.iloc[2,:],MC_045.iloc[1,:]],
                 columns=bank_names, 
                 index = ["GA CR+ MC","GA MtM MC"])
# Percentage Error NN
df_3 = pd.DataFrame([np.abs(100*(df_999.iloc[1,:]-df_2.iloc[0,:])/df_2.iloc[0,:])],
                 columns=bank_names, 
                 index = ["Percentage Error CR+ NN"])

df_4 = pd.DataFrame([np.abs(100*(df_999.iloc[2,:]-df_2.iloc[0,:])/df_2.iloc[0,:])],
                 columns=bank_names, 
                 index = ["Percentage Error CR+ approx"])
df_5 = pd.DataFrame([np.abs(100*(df_999.iloc[3,:]-df_2.iloc[1,:])/df_2.iloc[1,:])],
                 columns=bank_names, 
                 index = ["Percentage Error MtM NN"])

df_6 = pd.DataFrame([np.abs(100*(df_999.iloc[4,:]-df_2.iloc[1,:])/df_2.iloc[1,:])],
                 columns=bank_names, 
                 index = ["Percentage Error MtM approx"])
pd.concat([df_999,df_2,df_3,df_4,df_5,df_6])

1/1 [==============================] - 0s 38ms/step


C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\Capital Requirements of MDBs\Code_new\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - 0s 31ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.00,38.00,29.00,26.00,16.00,11.00,4.00,78.00,21.00,8.00,37.00
GA CR+ NN,15.00,7.61,8.44,9.45,8.38,17.46,19.51,4.17,13.16,16.04,7.69
GA CR+ approx,29.52,19.84,15.63,25.08,22.05,59.95,49.97,6.85,34.62,32.93,14.29
GA MtM NN,21.13,11.88,13.07,16.18,18.02,34.73,33.66,5.32,26.71,24.50,11.34
GA MtM approx,23.87,13.87,12.36,17.92,19.64,44.54,42.86,5.19,31.25,27.79,10.91
GA CR+ MC,12.46,7.07,8.00,9.24,11.89,20.38,27.13,4.35,15.23,15.78,8.76
GA MtM MC,18.71,10.34,12.48,13.32,18.76,25.85,36.44,6.38,25.95,22.75,11.20
Percentage Error CR+ NN,20.42,7.59,5.54,2.26,29.50,14.33,28.08,4.23,13.58,1.64,12.21
Percentage Error CR+ approx,136.90,180.64,95.32,171.46,85.49,194.18,84.18,57.48,127.33,108.66,63.09
Percentage Error MtM NN,12.91,14.93,4.71,21.49,3.97,34.37,7.63,16.68,2.94,7.71,1.21


## 2.) q = 99.9%, ELGD = 0.1

In [341]:
df_999_01 = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999,
               model_mtm = model_mtm_0999,
               model_actuarial = model_actuarial_0999)

df_2_01 = pd.DataFrame([MC_01.iloc[2,:],MC_01.iloc[1,:]],
                 columns=bank_names, 
                 index = ["GA CR+ MC","GA MtM MC"])
# Percentage Error NN
df_3_01  = pd.DataFrame([np.abs(100*(df_999_01 .iloc[1,:]-df_2_01 .iloc[0,:])/df_2_01 .iloc[0,:])],
                 columns=bank_names, 
                 index = ["Percentage Error CR+ NN"])

df_4_01  = pd.DataFrame([np.abs(100*(df_999_01 .iloc[2,:]-df_2_01 .iloc[0,:])/df_2_01 .iloc[0,:])],
                 columns=bank_names, 
                 index = ["Percentage Error CR+ approx"])
df_5_01  = pd.DataFrame([np.abs(100*(df_999_01 .iloc[3,:]-df_2_01 .iloc[1,:])/df_2_01 .iloc[1,:])],
                 columns=bank_names, 
                 index = ["Percentage Error MtM NN"])

df_6_01  = pd.DataFrame([np.abs(100*(df_999_01 .iloc[4,:]-df_2_01 .iloc[1,:])/df_2_01 .iloc[1,:])],
                 columns=bank_names, 
                 index = ["Percentage Error MtM approx"])
pd.concat([df_999_01 ,df_2_01 ,df_3_01 ,df_4_01 ,df_5_01 ,df_6_01 ])

1/1 [==============================] - 0s 53ms/step


C:\Users\jul_ses\OneDrive - National University of Singapore\Projects\Capital Requirements of MDBs\Code_new\misc_functions.py:130: RuntimeWarning: invalid value encountered in sqrt
  K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)*(1/(1-1.5*b))*(1+(M-2.5)*b)
C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


1/1 [==============================] - 0s 42ms/step


,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.00,38.00,29.00,26.00,16.00,11.00,4.00,78.00,21.00,8.00,37.00
GA CR+ NN,4.46,3.67,3.88,4.15,3.77,6.58,5.62,2.33,5.61,5.14,3.03
GA CR+ approx,20.35,13.93,10.68,17.51,14.56,41.84,29.58,4.57,24.81,22.77,9.61
GA MtM NN,15.46,8.86,8.38,12.19,12.36,25.64,20.49,3.66,22.19,18.69,6.94
GA MtM approx,17.93,10.63,9.03,13.61,13.62,34.11,25.42,3.60,26.85,21.42,7.59
GA CR+ MC,4.35,2.65,3.13,3.38,3.96,5.98,12.72,1.49,6.88,5.58,2.33
GA MtM MC,11.58,7.78,8.09,10.19,11.72,18.88,22.12,3.74,15.98,13.39,6.84
Percentage Error CR+ NN,2.52,38.66,24.03,22.72,4.81,10.02,55.79,56.28,18.45,7.95,29.96
Percentage Error CR+ approx,367.73,425.79,241.11,417.94,267.74,599.68,132.55,206.47,260.61,308.03,312.35
Percentage Error MtM NN,33.51,13.92,3.60,19.63,5.48,35.78,7.36,2.22,38.87,39.58,1.46
